In [1]:
import os
import gc
import sys
import time
import math
import json
import spacy
import random
import logging
import argparse
import itertools
import numpy as np
import _pickle as cPickle
import torch
os.chdir("../../src/shared/")
from classes import *
os.chdir("../../data/feature/")

In [2]:
logging.info('Loading training and dev data...')
with open('training_data',"rb") as f:
    training_data = cPickle.load(f)
with open("dev_data", 'rb') as f:
    dev_data = cPickle.load(f)

In [3]:
entity_mentions = []
event_mentions = []
for _, topic in training_data.topics.items():
    docs = topic.docs
    for _, doc in docs.items():
        for _, sent in doc.sentences.items():
            entity_mentions.extend(sent.gold_entity_mentions)
            event_mentions.extend(sent.gold_event_mentions)

In [4]:
entity_d = {}
event_d = {}

for entity in entity_mentions:
    tag = entity.gold_tag
    if tag in entity_d:
        entity_d[tag].append(entity)
    else:
        entity_d[tag] = [entity]

entity_d

{'HUM16236184328979740': [<classes.EntityMention at 0x7f7e12993c18>,
 'HUM16284637796168708': [<classes.EntityMention at 0x7f7e12993f28>,
 'LOC16235213289813758': [<classes.EntityMention at 0x7f7e12993fd0>,
 'HUM16236907954762763': [<classes.EntityMention at 0x7f7e129952e8>,
 'HUM17392465282479574': [<classes.EntityMention at 0x7f7e1299a828>,
 'Singleton_HUM_29_1_10ecb': [<classes.EntityMention at 0x7f7e1299a9e8>],
 'Singleton_HUM_27_1_10ecb': [<classes.EntityMention at 0x7f7e1299af28>],
 'TIM16236441119149059': [<classes.EntityMention at 0x7f7e1299d2b0>,
 'HUM16238359925764052': [<classes.EntityMention at 0x7f7e129a3f60>,
 'Singleton_LOC_8_1_12ecb': [<classes.EntityMention at 0x7f7e129b14e0>],
 'Singleton_HUM_14_1_13ecb': [<classes.EntityMention at 0x7f7e129b7550>],
 'Singleton_NON_13_1_13ecb': [<classes.EntityMention at 0x7f7e129b7dd8>],
 'Singleton_TIM_19_1_15ecb': [<classes.EntityMention at 0x7f7e1295d748>],
 'HUM16285687682097521': [<classes.EntityMention at 0x7f7e1296ca90>,
 'HUM

In [6]:
event_d = {}
for event in event_mentions:
    tag = event.gold_tag
    if tag in event_d:
        event_d[tag].append(event)
    else:
        event_d[tag] = [event]

event_d

{'ACT16236402809085484': [<classes.EventMention at 0x7f7e12993748>,
 'ACT16235311629112331': [<classes.EventMention at 0x7f7e129938d0>,
 'Singleton_ACT_31_1_10ecb': [<classes.EventMention at 0x7f7e12995fd0>],
 'Singleton_ACT_35_1_10ecb': [<classes.EventMention at 0x7f7e1299a438>],
 'ACT17393001387097507': [<classes.EventMention at 0x7f7e129b7048>,
 'Singleton_ACT_11_1_13ecb': [<classes.EventMention at 0x7f7e129b74e0>],
 'Singleton_ACT_46_1_14ecb': [<classes.EventMention at 0x7f7e129b8390>],
 'ACT16238560400862394': [<classes.EventMention at 0x7f7e129b85f8>,
 'Singleton_ACT_50_1_14ecb': [<classes.EventMention at 0x7f7e129c7ac8>],
 'ACT16238542452785616': [<classes.EventMention at 0x7f7e129c7da0>,
 'ACT17757159242353715': [<classes.EventMention at 0x7f7e129c7f98>,
 'Singleton_ACT_63_1_14ecb': [<classes.EventMention at 0x7f7e129cd240>],
 'ACT17757148989750169': [<classes.EventMention at 0x7f7e129cd438>,
 'NEG16237477263722686': [<classes.EventMention at 0x7f7e129cd7b8>,
 'Singleton_ACT_20

In [10]:
for i,v in event_d.items():
    for event in v:
        print(event.arg0)

('Tara Reid', '1_10ecb_0_3_4')
('she', '1_10ecb_3_7_7')
('her', '1_10ecb_3_20_20')
('Tara Reid', '1_11ecb_0_0_1')
('Tara Reid', '1_11ecb_2_1_2')
('Tara Reid', '1_12ecb_1_0_1')
('Tara Reid', '1_13ecb_1_2_3')
('Tara Reid', '1_14ecb_0_0_1')
('Tara Reid', '1_14ecb_1_9_10')
('Reid', '1_14ecb_2_32_32')
('She', '1_14ecb_5_1_1')
('herself', '1_14ecb_5_4_4')
('Tara Reid', '1_15ecb_0_4_5')
None
('she', '1_15ecb_1_18_18')
('Tara Reid', '1_17ecb_1_1_2')
('who', '1_18ecb_0_5_5')
('Tara Reid', '1_19ecb_0_0_1')
('actress', '1_19ecb_1_1_1')
("'Tara Reid", '1_19ecb_3_0_1')
('Tara Reid', '1_1ecb_1_11_12')
('actress', '1_1ecb_2_9_9')
('Tara Reid', '1_2ecb_0_3_4')
('Tara Reid', '1_2ecb_0_3_4')
('she', '1_2ecb_2_5_5')
('velvet-rope fixture', '1_3ecb_1_5_6')
('big screen', '1_4ecb_1_15_16')
('Tara Reid', '1_4ecb_2_12_13')
None
('Reid', '1_5ecb_3_0_0')
('Jack Ketsoyan', '1_6ecb_0_5_6')
('actress', '1_7ecb_0_5_5')
('actress', '1_7ecb_1_1_1')
('Tara Reid', '1_7ecb_2_1_2')
('she', '1_7ecb_4_7_7')
None
('Tara Re

In [8]:
import itertools
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

result = []
for i,v in event_d.items():
    if len(v)>1: # singletons
        subsets = findsubsets(v,2)
        for sebset in subsets:
            result.append(round(cos(sebset[0].head_embeddings, sebset[1].head_embeddings).tolist(),4))

In [19]:
l = len(result)
pos = [r for r in result if r>0.5]
len(pos)/l

0.9996670593658762

In [20]:
from collections import Counter
result = Counter(result).most_common()

In [21]:
result

[(0.9998, 348),
 (0.9999, 343),
 (0.9997, 210),
 (1.0, 162),
 (0.9996, 79),
 (0.7598, 36),
 (0.8039, 34),
 (0.7744, 34),
 (0.7996, 34),
 (0.7911, 33),
 (0.7925, 33),
 (0.7697, 33),
 (0.8513, 33),
 (0.8248, 32),
 (0.7916, 32),
 (0.7979, 32),
 (0.7991, 32),
 (0.7776, 32),
 (0.7546, 32),
 (0.7502, 32),
 (0.8059, 31),
 (0.7793, 31),
 (0.7877, 31),
 (0.8218, 31),
 (0.8211, 31),
 (0.7875, 31),
 (0.795, 31),
 (0.7782, 31),
 (0.8036, 31),
 (0.7766, 31),
 (0.8194, 30),
 (0.7721, 30),
 (0.7768, 30),
 (0.7918, 30),
 (0.757, 30),
 (0.7519, 30),
 (0.8031, 30),
 (0.8178, 30),
 (0.7896, 30),
 (0.788, 30),
 (0.7559, 30),
 (0.7914, 30),
 (0.8029, 30),
 (0.7491, 30),
 (0.7951, 30),
 (0.7878, 30),
 (0.8005, 30),
 (0.7706, 30),
 (0.7998, 30),
 (0.8152, 29),
 (0.7932, 29),
 (0.8024, 29),
 (0.8198, 29),
 (0.8092, 29),
 (0.8196, 29),
 (0.8133, 29),
 (0.7826, 29),
 (0.8319, 29),
 (0.7749, 29),
 (0.7743, 29),
 (0.7688, 29),
 (0.7978, 29),
 (0.768, 29),
 (0.7887, 29),
 (0.7816, 29),
 (0.8157, 28),
 (0.8185, 28)

## Testing the structure

In [8]:
topics = training_data.topics

In [16]:
topics

{'1_ecb': <classes.Topic at 0x7f4ecb40c860>,
 '3_ecb': <classes.Topic at 0x7f4ecac0ce80>,
 '4_ecb': <classes.Topic at 0x7f4ecaa32c18>,
 '6_ecb': <classes.Topic at 0x7f4eca5fec50>,
 '7_ecb': <classes.Topic at 0x7f4eca46f7f0>,
 '8_ecb': <classes.Topic at 0x7f4eca1ecfd0>,
 '9_ecb': <classes.Topic at 0x7f4ec9f4f780>,
 '10_ecb': <classes.Topic at 0x7f4ec9be5a90>,
 '11_ecb': <classes.Topic at 0x7f4ec99f8160>,
 '13_ecb': <classes.Topic at 0x7f4ec9771780>,
 '14_ecb': <classes.Topic at 0x7f4ec928ac50>,
 '16_ecb': <classes.Topic at 0x7f4ec90ea240>,
 '19_ecb': <classes.Topic at 0x7f4ec9034cf8>,
 '20_ecb': <classes.Topic at 0x7f4ec8c0ecf8>,
 '22_ecb': <classes.Topic at 0x7f4ec8ab77b8>,
 '24_ecb': <classes.Topic at 0x7f4ec87bda90>,
 '25_ecb': <classes.Topic at 0x7f4ec84aba90>,
 '26_ecb': <classes.Topic at 0x7f4ec7f36278>,
 '27_ecb': <classes.Topic at 0x7f4ec78ac5f8>,
 '28_ecb': <classes.Topic at 0x7f4ec6f5f5f8>,
 '29_ecb': <classes.Topic at 0x7f4ec68f4898>,
 '30_ecb': <classes.Topic at 0x7f4ec63cf1

In [19]:
all_docs = []
for i,topic in topics.items():
        all_docs.append(topic.docs)
all_docs[0]

{'1_10ecb': <classes.Document at 0x7f4ecb40cd30>,
 '1_11ecb': <classes.Document at 0x7f4ecb38c518>,
 '1_12ecb': <classes.Document at 0x7f4ecb372400>,
 '1_13ecb': <classes.Document at 0x7f4ecb310358>,
 '1_14ecb': <classes.Document at 0x7f4ecb3ffe80>,
 '1_15ecb': <classes.Document at 0x7f4ecb211a20>,
 '1_17ecb': <classes.Document at 0x7f4ecb1f9c50>,
 '1_18ecb': <classes.Document at 0x7f4ecb0fddd8>,
 '1_19ecb': <classes.Document at 0x7f4ecb07e2e8>,
 '1_1ecb': <classes.Document at 0x7f4ecb05d7f0>,
 '1_2ecb': <classes.Document at 0x7f4ecafea940>,
 '1_3ecb': <classes.Document at 0x7f4ecaf6b080>,
 '1_4ecb': <classes.Document at 0x7f4ecaf7b828>,
 '1_5ecb': <classes.Document at 0x7f4ecaee09e8>,
 '1_6ecb': <classes.Document at 0x7f4ecae70a58>,
 '1_7ecb': <classes.Document at 0x7f4ecada0128>,
 '1_8ecb': <classes.Document at 0x7f4ecacec9b0>,
 '1_9ecb': <classes.Document at 0x7f4ecac489e8>}

In [100]:
sentences = []
for i,doc in all_docs[0].items():
        sentences.append(doc.sentences)
sentences

[{0: <classes.Sentence at 0x7f4ecb40cda0>,
  3: <classes.Sentence at 0x7f4ecb3c32e8>},
 {0: <classes.Sentence at 0x7f4ecb38c550>,
  2: <classes.Sentence at 0x7f4ecb39dda0>},
 {1: <classes.Sentence at 0x7f4ecb372438>},
 {1: <classes.Sentence at 0x7f4ecb310390>},
 {0: <classes.Sentence at 0x7f4ecb3ffa90>,
  1: <classes.Sentence at 0x7f4ecb2ee1d0>,
  2: <classes.Sentence at 0x7f4ecb2fac18>,
  5: <classes.Sentence at 0x7f4ecb266a20>},
 {0: <classes.Sentence at 0x7f4ecb211a58>,
  1: <classes.Sentence at 0x7f4ecb23ae48>},
 {0: <classes.Sentence at 0x7f4ecb1f9c88>,
  1: <classes.Sentence at 0x7f4ecb144438>,
  2: <classes.Sentence at 0x7f4ecb144dd8>},
 {0: <classes.Sentence at 0x7f4ecb0fde10>},
 {0: <classes.Sentence at 0x7f4ecb07e320>,
  1: <classes.Sentence at 0x7f4ecb07ec88>,
  3: <classes.Sentence at 0x7f4ecb03e3c8>},
 {1: <classes.Sentence at 0x7f4ecb05d828>,
  2: <classes.Sentence at 0x7f4ecafc75c0>},
 {0: <classes.Sentence at 0x7f4ecafea978>,
  2: <classes.Sentence at 0x7f4ecaf92a58>},


In [101]:
event_mentions = []
entity_mentions = []
for i,sent in sentences[0].items():
    if i == 0:
        entity_mentions.append(sent.gold_entity_mentions)
        event_mentions.append(sent.gold_event_mentions)
        


In [102]:
event_mentions

[[<classes.EventMention at 0x7f4ecb41e6d8>,

In [103]:
entity_mentions

[[<classes.EntityMention at 0x7f4ecb41ea20>,

In [106]:
entity_mentions[0][0].get_tokens()

['Tara', 'Reid']

In [107]:
event_mentions[0][0].gold_tag

'ACT16236402809085484'